# Simulating a project portfolio
Lets get the simPy simulation package

In [ ]:
# @title Initialisations
!pip install simpy

import simpy, yaml
import pandas as pd
# Set the option to adjust display width to not limit the total width of the DataFrame display

# Set the maximum number of rows and columns to display
pd.set_option('display.max_rows', 100)  # Default is 60
pd.set_option('display.max_columns', 19)  # Default is 20

# Set the width and the maximum columns width
pd.set_option('display.width', 1000)  # Adjust accordingly
pd.set_option('display.max_colwidth', 100)  # Adjust accordingly
# Set display precision to 2 decimal places
pd.set_option('display.precision', 2)


from google.colab import data_table
data_table.enable_dataframe_formatter()
data_table._DEFAULT_FORMATTERS[float] = lambda x: f"{x:.2f}"

Lets get the **helper** functions we need.

In [ ]:
# @title Helper functions
ALL_MONTHS = [
 "jan",
 "feb",
 "mar",
 "apr",
 "may",
 "jun",
 "jul",
 "aug",
 "sep",
 "oct",
 "nov",
 "dec",
]


def get_current_month(start_month='apr', month=0):
	# Start month for the financial year is always April, which is the 4th month, index 3 in ALL_MONTHS
	# Since env.now starts from 1 for April, subtract 1 to align with zero-based indexing
	elapsed_months_adjusted = month

	# Calculate the index of the current month
	# April's index (3) is used as a fixed starting point for financial year starting in April
	current_month_index = (3 + elapsed_months_adjusted) % len(ALL_MONTHS)

	# Return the name of the current month
	return ALL_MONTHS[current_month_index]


def printtimestamp(env):
	start_month = 'apr'
	month = get_current_month(start_month, env.now - 1)
	print(f"\nMonth: {env.now} ({month})")

def pivotbudget(db):
  df = db.groupby(['step', 'item']).agg({'budget': 'sum'}).reset_index()
  # Pivot the DataFrame
  pf = df.pivot_table(index='item', columns='step', values='budget', aggfunc='sum', fill_value=0)
  pf = pf.iloc[::-1]

  return(pf)

def parseYAML(yamltext):

    # |SubFunction to map 'cls' strings to function objects
    def map_cls_strings_to_objects(data):
      if isinstance(data, list):
          for index, item in enumerate(data):
              data[index] = map_cls_strings_to_objects(
                  item)  # Assign the returned value in case of lists
      elif isinstance(data, dict):
          for key, value in data.items():
              if key == 'cls' and isinstance(value, str):
                  # Attempt to replace string with class from globals
                  data[key] = globals().get(value, value)  # Fallback
              else:
                  data[key] = map_cls_strings_to_objects(
                      value)  # Recurse into nested structures
      return data  # Important: return the modified data

    data = yaml.safe_load(yamltext)
    map_cls_strings_to_objects(data)
    return data

Get the staff classes and associated Global variables

In [ ]:
# @title Staff Classes
#from functions import *

NIRATE = 0.138
NITHRESHOLD = 175
EMPLOYERPENSIONRATE = 0.09
PENSIONFTETHRESHOLD = 0.2
REALLIVINGWAGE = 12

#Need to change all the below to **kwargs

class worker:
    def __init__(self,**kwargs):
        self.position = kwargs.get('position','undesignated')
        self.name = kwargs.get('name','staff member')
        self.age = kwargs.get('age',49)
        self.department = kwargs.get('department','unspecified')
        self.mobilephone = kwargs.get('mobilephone','not assigned')
        self.linemanagerrate = kwargs.get('linemanagerrate',0)

        self.fte_salary = kwargs.get('salary',0)
        self.fte = kwargs.get('fte',1)
        self.salary = self.fte * self.fte_salary

    def info(self):
        for attr, value in self.__dict__.items():
            print(f"{attr} : {value}")

    def getbreakdown(self, month):
        data=[]
        salary = self.getMonthSalaryCost(month)
        data.append({'step':month,'item': 'salary', 'budget':salary})
        data.append({'step':month,'item': 'ni', 'budget':self.getNI(salary)})
        data.append({'step':month,'item': 'pension', 'budget':self.getPension(salary, self.fte)})

        return(data)

    def getSalaryCost(self):
        salary = self.salary
        monthlysalary = salary / 12
        monthlycost = (
            monthlysalary
            + self.getNI(monthlysalary)
            + self.getPension(monthlysalary, self.fte)
        )
        annualcost = monthlycost * 12
        annualcost = (annualcost)
        return annualcost

    def getMonthSalaryCost(self, month):
        return (self.getSalaryCost() / 12)

    def getNI(self, monthlySalary):
        niRate = NIRATE
        threshold = NITHRESHOLD
        monthlyThreshold = threshold / 7 * 365 / 12

        if self.salary > monthlyThreshold:
            ni = max(0, (monthlySalary - monthlyThreshold)) * niRate
        else:
            ni = 0
        return (ni)

    def getPension(self, salary, fte):
        pensionRate = EMPLOYERPENSIONRATE
        if fte > PENSIONFTETHRESHOLD:
            pension = salary * pensionRate
        else:
            pension = 0

        return pension


class visitorcentre_staff(worker):
    def __init__(self,**kwargs):
        kwargs['department'] = 'visitor centres'
        super().__init__(**kwargs)

    def test(self):
        pass


class seasonal_assistant(visitorcentre_staff):
    def __init__(
        self,
        name="seasonal",
        fte_salary=0,
        working_pattern={},
        position="seasonal_assistant",**kwargs
    ):
        super().__init__(position=position, fte_salary=fte_salary, name=name, **kwargs)
        self.working_pattern = working_pattern or {}
        self.pad_working_pattern()

    def pad_working_pattern(self):
        for month in ALL_MONTHS:
            self.working_pattern.setdefault(month, 0)

    def getSalaryCost(self):
        annualtotal = 0
        working_pattern = self.working_pattern

        for month in working_pattern:
            annualtotal += self.getMonthSalaryCost(month)

        annualtotal = (annualtotal)
        return annualtotal

    def getMonthSalaryCost(self, month):
        monthly_fte_salary = self.fte_salary / 12
        working_pattern = self.working_pattern

        monthlyfte = working_pattern[month]
        month_salary = monthly_fte_salary * monthlyfte

        ni = self.getNI(month_salary)
        pension = self.getPension(month_salary, monthlyfte)
        return (month_salary + ni + pension)


class teacher_naturalist(visitorcentre_staff):
    def __init__(
        self,
        hourly_salary=REALLIVINGWAGE,
        hours_worked=10,
        name="TN",
        working_pattern={},
        position="Teacher Naturalist", **kwargs
    ):
        self.hours_worked = hours_worked
        self.hourly_salary = float(hourly_salary) #WHY IS FLOAT NEEDED?
        super().__init__(position=position, name=name, **kwargs)
        self.fte_salary = self.calculateFTESalary()
        self.fte = self.calculateFTE()

    def totalpossiblehrs(self):
        tot = 35 * (52 - 5.6)
        return tot

    def calculateFTE(self):
        fte = self.hours_worked / self.totalpossiblehrs()
        return (fte, 2)

    def calculateFTESalary(self):
        sal = self.hourly_salary
        sal = sal * self.totalpossiblehrs()
        return (sal)

    def getMonthSalaryCost(self, month):
        return 0

    def getSalaryCost(self):
        sal = self.hourly_salary * self.hours_worked
        return (sal)




Lets get the **project** classes that specify which projects we can instantiate. The root class is called **project** and its subclasses are combined in different ways to create the other projects.

In [ ]:
# @title Project Classes
#from functions import *
#from staff import worker


class project:

	def __init__(self, portfolio, env, **kwargs):
		self.kwargs = kwargs
		self.name = kwargs.get('name','New Project')
		self.term = kwargs.get('term',0)
		self.directcosts = kwargs.get('directcosts',[])
		self.env = env
		self.portfolio = portfolio
		self.startstep = env.now
		self.consolidated_account = portfolio.consolidated_account
		self.budget = kwargs.get('budget', 0)

		self.policies=[]
		policies = kwargs.get('policies')

		if policies:
			for policy in policies:
				cls=globals().get(policy['policy'])
				if callable(cls):
					self.policies.append(cls(self.env, self, **policy))

		self.staff = []
		staffing = kwargs.get('staffing',[])
		for person in staffing:
			self.addstaff(worker(**person))

		self.costs_thismonth = 0
		self.income_thismonth = 0
		self.cost = 0
		self.income = 0
		self.env.process(self.start())

	def calculate(self, step):
		dcosts = self.getdirectcosts(step)
		directcost = sum([d['budget'] for d in dcosts if 'budget' in d])

		self.costs_thismonth += self.getsalarycosts(step) + directcost  #self.budget / self.term
		self.income_thismonth += 0 #self.costs_thismonth * 1.5 #Very simply estimation of income_thismonth generated for the base class

		return

	def getdirectcosts(self,step):
		directcosts = self.directcosts
		costs = []

		for directcost in directcosts:
			freq = directcost.get('frequency','oneoff')
			applystep = directcost.get('step',0)
			item = directcost.get('item','unspecified')
			cost = directcost.get('cost',0)

			if freq=='monthly' or (freq=='oneoff' and applystep==step) or (freq=='annual' and (step-applystep) % 12 == 0):
				cost=cost
			else:
				cost=0

			costs.append({'step':step, 'item':item, 'budget':cost})

		return costs

	def getstaffcosts(self, step=None):

		# Assuming FullCostRecovery is a class you have defined elsewhere
		fcr_policy = next((policy for policy in self.policies if isinstance(policy, FullCostRecovery)), None)

		def getstep(step):
			stepregister = []
			for person in self.staff:

				# Extend register with breakdown, directly setting 'name' for each entry
				breakdown = person.getbreakdown(step)
				for entry in breakdown:
						entry["name"] = person.name
				register.extend(breakdown)

				# Assuming fcr_policy is not None and getfcr method returns a list of dicts
				if fcr_policy is not None:
						fcr_entries = fcr_policy.getfcr(person, step)
						for entry in fcr_entries:
								entry["name"] = person.name

						stepregister.extend(fcr_entries)

			return(stepregister)

		register = []
		if step is not None:
			register.extend(getstep(step))
		else:
			for step in range(self.term):
				register.extend(getstep(step))


		db = pd.DataFrame(register)
		return db

	def getbudget(self):
		budget=[]
		for i in range(self.term):
			directcosts = self.getdirectcosts(i)
			budget.extend(directcosts)

			for staff in self.staff:
				budget.extend(staff.getbreakdown(i))

		for policy in self.policies:
			if hasattr(policy, 'getbudget'):
				if callable(policy.getbudget):
					budget.extend(policy.getbudget())


		df = pd.DataFrame(budget) #keep unpivoted while still working on the dataframe
		return(df)

	def getbudgetadjusted(self):
		df = self.getbudget()
		if 'step' in df.columns:
			df['step'] = df['step'] + self.startstep
		return(df)


	def getsalarycosts(self, step):
		cost = 0
		for worker in self.staff:
			cost += worker.getMonthSalaryCost(step)
			#print(f'worker {worker.name} costs {worker.getMonthSalaryCost(step)} per month')

		return (cost)

	def addstaff(self, staff):
		self.staff.append(staff)

	def sweep_policies(self, step):
		for policy in self.policies:
			policy.calculate(step)


	def start(self):
		for i in range(self.term):
			self.income_thismonth = self.costs_thismonth = 0
			self.calculate(i)
			self.sweep_policies(i)
			self.income += self.income_thismonth
			self.cost += self.costs_thismonth

			consoldacc = self.portfolio.consolidated_account

			consoldacc.update({
			 'type': 'expenditure',
			 'title': 'project costs',
			 'project': self.name,
			 'amount': self.costs_thismonth
			})
			consoldacc.update({
			 'type': 'income',
			 'title': 'project income',
			 'project': self.name,
			 'amount': self.income_thismonth
			})


			yield self.env.timeout(1)

		printtimestamp(self.env)
		print(
		 f"Project {self.name} cost {self.cost:.2f} and generated {self.income:.2f} with budget {self.budget:.2f}"
		)


class visitorexperience_project(project):

	def __init__(self, portfolio, env, **kwargs):
		project.__init__(self,portfolio, env,**kwargs)
		#from vc import montrose
		self.vcs = [montrose]

	def calculate(self, step):
		super().calculate(step)

		#from functions import get_current_month
		month = get_current_month(start_month='apr', month=self.env.now)
		sitecosts = sum([vc.monthlycosts(month) for vc in self.vcs])
		siteincome = sum([vc.monthlyincome(month) for vc in self.vcs])

		sitecostsarray = {'item':'sitecosts', 'step': step, 'cost': sitecosts}
		self.directcosts.append(sitecostsarray)
		self.income_thismonth += siteincome
		self.costs_thismonth += sitecosts
		return









The **vc** classes have specific functionality for visitor centres in different configurations.

In [ ]:
# @title VC Classes
class visitorcentre:
    def __init__(self, hasShop=False, hasBuilding=True, name="Visitor Centre"):
        self.staffroll = []
        self.shop = shop() if hasShop else None
        self.building = building() if hasBuilding else None
        self.name = name

    def info(self):
        for attr, value in self.__dict__.items():
            print(f"{attr} : {value}")

    def monthlycosts(self, month='apr'):
        return(self.staffingcosts(month) + self.building.costs(month) + self.shop.costs(month))


    def monthlyincome(self, month='apr'):
        income_thismonth = self.shop.sales(month)
        return(income_thismonth)

    def staffingcosts(self, month=None):
        if month:
            tot = sum([person.getMonthSalaryCost(month) for person in self.staffroll])
        else:
            tot = sum([person.getSalaryCost() for person in self.staffroll])
        return tot

    def runningcosts(self, month):
        if month:
            tot = self.building.costs(month)
        else:
            tot = self.building.costs()

        return tot

    def annualcosts(self):
        return self.staffingcosts() + self.building.costs() + self.shop.costs()


class shop:
    def __init__(self):
        self.stock = []

    def costs(self, month=None):
        stockpurchase = 2000
        return stockpurchase if not month else stockpurchase/12

    def sales(self, month=None):
        sales = 2500
        admissions = 3000
        tot = sales+admissions
        return tot if month else tot*12

class building:
    def __init__(self):
        pass

    def costs(self, month=None):
        costs = (
            self.calculatemaintenance()
            + self.calculateutilities()
            + self.calculatecleaning()
        )

        if month:
            costs /= 12

        return (costs)

    def calculatemaintenance(self):
        return 10000

    def calculateutilities(self):
        return 5000

    def calculatecleaning(self, daysperyear=50, dayrate=150):
        return daysperyear * dayrate

Lets set up Montrose VC

In [ ]:
# @title Set up VCs
yamltext = f"""
- cls: visitorcentre_staff
  position: Site Manager
  fte_salary: 24000
  name: Melvyn Turok
  mobilephone: true

- cls: visitorcentre_staff
  position: VC Assistant
  fte_salary: 19000
  name: Janet Cushing
  fte: 0.8

- cls: seasonal_assistant
  name: Jason Scott
  fte_salary: 19000
  working_pattern:
    feb: 1
    mar: 1
    apr: 1
    may: 1

- cls: teacher_naturalist
  name: Alison Knight
  hourly_salary: {REALLIVINGWAGE *1.05}
  hours_worked: 30
"""
staff=[]
for staffmember in parseYAML(yamltext):
  cls = staffmember['cls']
  staff.append(cls(**staffmember))

montrose = visitorcentre(hasShop=True, name="Montrose Visitor Centre")
[montrose.staffroll.append(person) for person in staff if person.department == "visitor centres"]

[None, None, None, None]

Let's set up the Consolidated Account and the Portfolio that will hold all the projects.

In [ ]:
# @title FCR data
yamltext = f"""
- item: Recruitment
  daysperfte: 2
  dayrate: 490
  frequency: oneoff

- item: Finance set-up & Budget import
  daysperfte: {5.25/7}
  dayrate: 420
  frequency: oneoff

- item: Line Management
  daysperfte: 2
  dayrate: 0
  frequency: monthly

- item: ICT procurement & setup
  daysperfte: 1
  dayrate: 350
  frequency: oneoff

- item: ICT support
  daysperfte: {3/7}
  dayrate: 350
  frequency: annual

- item: HR (payroll, timesheets, performance management)
  daysperfte: {1/7}
  dayrate: 490
  frequency: monthly
"""
FCRDATA = parseYAML(yamltext)

In [ ]:
# @title Support services

yamltext = f"""
- item: Map production
  dayrate: 245
  daysperunit: {2/7}
  units: 0

- item: Leaflet production
  dayrate: 490
  daysperunit: 2
  units: 0

- item: Bespoke Leaflet production
  dayrate: 490
  daysperunit: {28/7}
  units: 0

- item: PR support
  dayrate: 315
  daysperunit: 1
  units: 0

- item: Single Page Website
  dayrate: 490
  daysperunit: {21/7}
  units: 0

- item: Multiple Page Website
  dayrate: 490
  daysperunit: {49/7}
  units: 0

- item: Interpretation panel production
  dayrate: 490
  daysperunit: {21/7}
  units: 0

- item: Bespoke panel production
  dayrate: 490
  daysperunit: {28/7}
  units: 0

- item: Support with grant claims (Fundraising)
  dayrate: 385
  daysperunit: {3.5/7}
  units: 0

- item: Support with grant claims (Finance)
  dayrate: 420
  daysperunit: {3.5/7}
  units: 0
"""
SUPPORTDATA = parseYAML(yamltext)

In [ ]:
# @title Central accounts
class ConsolidatedAccount:

    def __init__(self, env):
        self.env = env
        self.total_capital = 0
        self.total_payments = 0
        self.total_income = 0
        self.balance = 0
        #self.projects = []
        self.register = []

    def update(self, transaction):
        transaction['amount'] = float(transaction['amount'])
        if transaction['type'] == 'expenditure':
            self.total_payments += transaction['amount']
        if transaction['type'] == 'income':
            self.total_income += transaction['amount']
            transaction['amount'] = -transaction['amount']

        self.balance = self.total_income - self.total_payments

        transaction['date'] = self.env.now
        transaction['balance'] = self.balance
        self.register.append(transaction)

    def report(self):
        print(
            f"Consolidated Account Report: Payments to date: {self.total_payments:.2f}, Income to date: {self.total_income:.2f}, Balance: {self.balance:.2f}"
        )


class Portfolio:

    def __init__(self, env, name='My Portfolio'):
        self.env = env
        self.name = name
        self.consolidated_account = ConsolidatedAccount(env)
        self.projects = []

    def counter(self):
        for i in range(1, 31):
            month = get_current_month(start_month='apr', month=self.env.now)
            print(f"\nMonth: {i} {month}")
            yield self.env.timeout(1)

    def set_event(self, event):
        e = self.env.event()
        e.details = event
        yield self.env.timeout(event["time"])
        printtimestamp(
            self.env)  # Assuming this is a function you have defined elsewhere
        print(f'Event {event["message"]} succeeds')
        e.succeed()
        self.env.process(self.create_project(**event))

    def set_portfolio(self, events):
        for event in events:
            self.env.process(self.set_event(event))

    def getbudget(self):
        data = {
          'item': [],
          'step': [],
          'budget': []
        }
        consol_budget = pd.DataFrame(data)

        for prj in self.projects:
          budget = prj.getbudgetadjusted()
          consol_budget = pd.concat([consol_budget, budget], ignore_index=True)

        return(consol_budget)


    def list_projects(self):
        projects = self.projects
        data = []
        for prj in projects:
            data.append({
                k: v
                for k, v in prj.__dict__.items()
                if isinstance(v, (str, int, float, bool))
            })
        df = pd.DataFrame(data)

        return df  # Return the DataFrame instead of printing it

    def run(self, until):
        #self.env.process(self.counter())
        self.env.run(until=until)

    def list_transactions(self):
        transactions = self.consolidated_account.register
        # Convert the list of dictionaries to a DataFrame
        df = pd.DataFrame(transactions)
        # Print the DataFrame as a table
        #df.head(40)
        self.consolidated_account.report()
        return(df)


    def create_project(self, cls=project, **kwargs):
        prj = cls(self, self.env, **kwargs)  # Use self.env here

        self.projects.append(prj)

        staff_names = ', '.join(person.name for person in prj.staff)
        print(
            f"Project {prj.name} created with budget {prj.budget:.2f} and assigned staff {staff_names}"
        )
        yield self.env.timeout(1)  # Use self.env here

    def finance(self, term, capital, rate=0.05):
        repayment = capital / term
        account = capital
        print(f'New capital received {capital}')
        self.consolidated_account.update({
            'type': 'income',
            'title': 'finance capitalisation',
            'project': 'headoffice',
            'amount': capital
        })
        totpay = 0
        for i in range(term):
            interest = rate * account
            account = account - repayment
            payment = repayment + interest
            totpay += payment
            self.consolidated_account.update({
                'type': 'expenditure',
                'title': 'finance servicing',
                'project': 'headoffice',
                'amount': payment
            })
            #print(f"{i+1} out of {term} paid {payment:.2f}, remaining loan: {account:.2f}")
            yield self.env.timeout(1)

        printtimestamp(self.env)
        print(f"Finance: Final account {account:.2f}, total paid {totpay:.2f}")




In [ ]:
# @title Policies

class Policy():
  def __init__(self, env, prj, **kwargs):
    self.env = env
    self.prj = prj

class FullCostRecovery(Policy):
  def __init__(self, env, prj, **kwargs):
    super().__init__(env, prj, **kwargs)
    self.supports = kwargs.get('supports',[])

    self.fcr = self.getfcrdata()
    self.register = []

  def getfcrdata(self):
    fcr=[]
    for item in FCRDATA:
      fcr.append(item)

    return fcr

  def checksupports(self, step):
    totcost = 0
    for support in self.supports:
      matching_dictionaries = [d for d in SUPPORTDATA if d.get('item') == support['item']]
      support['step'] = support['step'] if 'step' in support else 0
      if support['step'] == step and len(matching_dictionaries)>0:
        item = support['item']

        lookup = matching_dictionaries[0]
        cost = support['units'] * lookup['dayrate'] * lookup['daysperunit']
        entry = {'item':item, 'budget':cost, 'step':step}
        totcost += cost

        self.register.append(entry)


      #adjust to store support items to register and

    return totcost

  def getfcr(self, person, step):
    register=[]
    linemanagerrate = person.linemanagerrate
    fte = person.fte

    for item in self.fcr:
      itemname = item['item']
      daysperfte = item['daysperfte'] * fte
      dayrate = linemanagerrate if itemname == 'Line Management' else item['dayrate']
      frequency = item['frequency']

      cost = person.fte*daysperfte*dayrate
      if frequency == 'oneoff':
        cost = cost if step==0 else 0

      if frequency == 'monthly':
        cost = cost

      if frequency == 'annual':
        cost = cost if step % 12 == 0 else 0

      register.append({'step': step, 'item': itemname, 'budget': cost})
    return(register)

  def calcfcr(self, person, step):
    register = self.getfcr(person, step)
    self.register.extend(register)
    result = (sum(item['budget'] for item in register if 'budget' in item))
    return((result))

  def calculate(self, step):
    totalcost=0
    for person in self.prj.staff:
      totalcost += self.calcfcr(person, step)

    prj=self.prj
    prj.consolidated_account.update({
        'type': 'expenditure',
        'title': 'full cost recovery (staff)',
        'project': prj.name,
        'amount': totalcost
    })

    totalcost += self.checksupports(step)
    prj.costs_thismonth+=totalcost

  def getbudget(self):
    return(self.register)

class Grant(Policy):
  def __init__(self, env, prj, **kwargs):
    super().__init__(env, prj, **kwargs)
    self.amount = kwargs.get('amount',0)
    self.fund = kwargs.get('fund','unspecified')
    self.startstep = kwargs.get('step', 0)
    self.register = []

  def calculate(self, step):
    prj = self.prj
    amount = self.amount
    if step == self.startstep:
      prj.income_thismonth += amount
      prj.consolidated_account.update({
          'type': 'income',
          'title': f'grant from {self.fund}',
          'project': prj.name,
          'amount': amount
      })
      self.register.append({'item': f'{self.fund} grant', 'step': step, 'budget': -amount})

  def getbudget(self):
    return(self.register)


class Subsidy(Policy):

  def calculate(self, step):
    payment=100000
    prj=self.prj
    prj.income_thismonth+=payment
    prj.consolidated_account.update({
        'type': 'income',
        'title': 'government subsidy',
        'project': prj.name,
        'amount': payment
    })

class Rename(Policy):
  def calculate(self, step):
    prj=self.prj
    prj.name = f'Fancy project in step {step}'

class Finance(Policy):
  def __init__(self, env, prj, **kwargs):
    super().__init__(env, prj, **kwargs)
    #kwargs = prj.kwargs
    self.term = kwargs.get('term',prj.term)
    self.account = self.capital = kwargs.get('capital',0)
    self.rate = kwargs.get('rate',0)
    self.consolidated_account = prj.consolidated_account

    self.totpay = 0
    print(f'New capital received {self.capital}')
    self.consolidated_account.update({
		 'type': 'income',
		 'title': 'finance capitalisation',
		 'project': 'headoffice',
		 'amount': self.capital
		})

  def calculate(self, step):
    repayment = self.capital / self.term
    interest = self.rate * self.account
    self.account -= repayment
    payment = repayment + interest
    self.totpay += payment
    self.consolidated_account.update({
		    'type': 'expenditure',
		    'title': 'finance servicing',
		    'project': 'headoffice',
		    'amount': payment
		})

    if step == self.term-1:
      self.finalize()

  def finalize(self):
    printtimestamp(self.env)
    print(
    f"Finance: Final account {self.account:.2f}, total paid {self.totpay:.2f}")


class CarbonFinancing(Policy):

  def __init__(self, env, prj, **kwargs):
    #kwargs = prj.kwargs
    self.prj = prj
    self.budget = prj.budget
    self.investment = kwargs.get('investment')
    self.tree_planting_cost_per_unit = kwargs.get('tree_planting_cost_per_unit')
    self.carbon_credit_per_unit = kwargs.get('carbon_credit_per_unit')
    self.trees_planted = self.calculate_trees_planted()
    self.carbon_credits_generated = self.calculate_carbon_credits()
    self.prj.consolidated_account.update({
			 'type': 'expenditure',
			 'title': 'capital cost tree planting',
			 'project': self.prj.name,
			 'amount': self.investment-self.budget
		})
    print(f'Trees planted: {self.trees_planted:.0f} will generate {self.calculate_carbon_credits():.0f} carbon credits over 40 years worth £{self.calculate_carbon_income():.2f}')

  def calculate_trees_planted(self):
		# Assumes all investment goes to tree planting after removing the budget
    return (self.investment-self.budget) / self.tree_planting_cost_per_unit

  def calculate_carbon_credits(self):
		# Calculates carbon credits based on the number of trees planted

    unitpertreelifetime=1.1 #average tree sequesters 1.1 tonnes over 40 years lifetime.
    return self.trees_planted * unitpertreelifetime

  def calculate_carbon_income(self):
    return self.calculate_carbon_credits()*self.carbon_credit_per_unit

  def report(self):
		# Reports on the carbon financing arrangement
    return {
		 "investment": self.investment,
		 "trees_planted": self.trees_planted,
		 "carbon_credits_generated": self.carbon_credits_generated
		}

  def calculate(self, step):
    # self.costs_thismonth = self.getsalarycosts(step) + self.directcosts  #self.budget / self.term

    if step==0:
      carbonincome = self.investment #Grab the full investment in the first year only
    else:
      carbonincome = 0

    self.prj.income_thismonth += carbonincome
    return

Now we specify the actual project data we will use in YAML format for easy readability and tweakability.

In [ ]:
# @title Project data { run: "auto" }
InterestRate = 0.05 # @param {type:"number"}
CapitalBorrowing = 80000 # @param {type:"number"}
project_data = f"""
- time: 1
  message: capitalisation
  cls: project
  name: Finance raising initiative
  term: 24
  budget: 3000

  directcosts:
  - item: Laptop
    cost: 1000
    frequency: oneoff

  policies:
  - policy: Finance
    term: 24
    capital: {CapitalBorrowing}
    rate: {InterestRate}
  - policy: FullCostRecovery
    supports:
    - item: Map production
      units: 2

  staffing:
  - name: Clerica
    salary: 28000
    fte: 0.05
    linemanagerrate: 500
  - name: Prunella
    salary: 19500
    fte: 0.10
    linemanagerrate: 250

- time: 1
  message: new project start
  cls: project
  term: 24
  budget: 90000
  name: Starbright
  policies:
  - policy: Grant
    amount: 45000
    fund: NLHF
    step: 0
  - policy: FullCostRecovery
    supports:
    - item: Leaflet production
      step: 3
      units: 1
    - item: Map production
      step: 6
      units: 2
  staffing:
  - name: Bubu
    salary: 20000
    linemanagerrate: 200
  - name: Olaf
    salary: 24000
    linemanagerrate: 200

- time: 12
  message: new project start
  cls: project
  term: 6
  budget: 25000
  name: Starbright lite
  staffing:
  - name: Hula
    salary: 50000
    linemanagerrate: 600

- time: 1
  message: new project start
  cls: visitorexperience_project
  term: 24
  budget: 250000
  name: Visitor Experience
  staffing:
  - name: Carline
    salary: 33000
    linemanagerrate: 600
  - name: Nat
    salary: 22000
    linemanagerrate: 400
    fte: 0.4
  - name: Jake
  policies:
  - policy: FullCostRecovery

- time: 3
  message: new restoration project
  cls: project
  name: New Planting
  term: 24
  budget: 15000
  directcosts:
  - item: planting
    cost: 500
    frequency: monthly
  policies:
  - policy: Grant
    amount: 20000
    fund: Forestry Grant Scheme
  - policy: CarbonFinancing
    investment: 100000
    tree_planting_cost_per_unit: 1
    carbon_credit_per_unit: 30
  - policy: FullCostRecovery
  staffing:
  - name: Buzz
    salary: 27000
    fte: 0.05
    linemanagerrate: 350
"""

Finally let's read in the project data and start the simulation

In [ ]:
# @title Run Simulation { run: "auto" }
steps = 50 # @param {type:"slider", min:1, max:50, step:1}
env = simpy.Environment()

# Instantiate portfolio and set up events
my_events = parseYAML(project_data)

portfolio = Portfolio(env)
portfolio.set_portfolio(my_events)

# Run the simulation
portfolio.run(until=steps)


Month: 1 (apr)
Event capitalisation succeeds
New capital received 80000
Project Finance raising initiative created with budget 3000.00 and assigned staff Clerica, Prunella

Month: 1 (apr)
Event new project start succeeds
Project Starbright created with budget 90000.00 and assigned staff Bubu, Olaf

Month: 1 (apr)
Event new project start succeeds
Project Visitor Experience created with budget 250000.00 and assigned staff Carline, Nat, Jake

Month: 3 (jun)
Event new restoration project succeeds
Trees planted: 85000 will generate 93500 carbon credits over 40 years worth £2805000.00
Project New Planting created with budget 15000.00 and assigned staff Buzz

Month: 12 (mar)
Event new project start succeeds
Project Starbright lite created with budget 25000.00 and assigned staff Hula

Month: 18 (sep)
Project Starbright lite cost 30070.38 and generated 0.00 with budget 25000.00

Month: 24 (mar)
Finance: Final account -0.00, total paid 130000.00

Month: 25 (apr)
Project Finance raising initiati

In [ ]:
portfolio.list_projects()

,name,term,startstep,budget,costs_thismonth,income_thismonth,cost,income
0,Finance raising initiative,24,1,3000,287.54,0,8065.31,0
1,Starbright,24,1,90000,5232.79,0,130597.00,45000
2,Visitor Experience,24,1,250000,7592.26,5500,186415.50,132000
3,New Planting,24,3,15000,614.42,0,14751.06,120000
4,Starbright lite,6,12,25000,5011.73,0,30070.38,0


In [ ]:

portfolio.list_transactions()

Consolidated Account Report: Payments to date: 651327.63, Income to date: 442000.00, Balance: -209327.63


,type,title,project,amount,date,balance
0,income,finance capitalisation,headoffice,-80000.00,1,80000.00
1,expenditure,finance servicing,headoffice,7333.33,1,72666.67
2,expenditure,full cost recovery (staff),Finance raising initiative,30.81,1,72635.85
3,expenditure,project costs,Finance raising initiative,1449.98,1,71185.88
4,income,project income,Finance raising initiative,-0.00,1,71185.88
...,...,...,...,...,...,...
323,expenditure,project costs,New Planting,614.42,25,-208711.28
324,income,project income,New Planting,-0.00,25,-208711.28
325,expenditure,full cost recovery (staff),New Planting,1.93,26,-208713.20
326,expenditure,project costs,New Planting,614.42,26,-209327.63


In [ ]:
prj = portfolio.projects[1]
print(prj.name)

db = prj.getbudget()
# # Pivot the DataFrame
db.pivot_table(index='item', columns='step', values='budget', aggfunc='sum', fill_value=0)

Starbright


step,0,1,2,3,4,5,6,7,8,...,15,16,17,18,19,20,21,22,23
item,,,,,,,,,,,,,,,,,,,
Finance set-up & Budget import,630.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"HR (payroll, timesheets, performance management)",140.00,140.00,140.00,140.00,140.00,140.00,140.00,140.00,140.00,...,140.00,140.00,140.00,140.00,140.00,140.00,140.00,140.00,140.00
ICT procurement & setup,700.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ICT support,300.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Leaflet production,0.00,0.00,0.00,980.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Line Management,800.00,800.00,800.00,800.00,800.00,800.00,800.00,800.00,800.00,...,800.00,800.00,800.00,800.00,800.00,800.00,800.00,800.00,800.00
Map production,0.00,0.00,0.00,0.00,0.00,0.00,140.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
NLHF grant,-45000.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Recruitment,1960.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [ ]:
db = portfolio.getbudget()
pivotbudget(db)

step,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0
item,,,,,,,,,,,,,,,,,,,
sitecosts,2041.67,2041.67,2041.67,2041.67,2041.67,2041.67,2041.67,2041.67,2041.67,...,2041.67,2041.67,2041.67,2041.67,2041.67,2041.67,2041.67,0.00,0.00
salary,8643.35,8643.35,8755.85,8755.85,8755.85,8755.85,8755.85,8755.85,8755.85,...,8755.85,8755.85,8755.85,8755.85,8755.85,8755.85,8755.85,112.50,112.50
planting,0.00,0.00,500.00,500.00,500.00,500.00,500.00,500.00,500.00,...,500.00,500.00,500.00,500.00,500.00,500.00,500.00,500.00,500.00
pension,752.78,752.78,752.78,752.78,752.78,752.78,752.78,752.78,752.78,...,752.78,752.78,752.78,752.78,752.78,752.78,752.78,0.00,0.00
ni,734.51,734.51,734.51,734.51,734.51,734.51,734.51,734.51,734.51,...,734.51,734.51,734.51,734.51,734.51,734.51,734.51,0.00,0.00
Recruitment,4089.05,0.00,2.45,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
NLHF grant,-45000.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Map production,140.00,0.00,0.00,0.00,0.00,0.00,140.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Line Management,2135.50,2135.50,2137.25,2137.25,2137.25,2137.25,2137.25,2137.25,2137.25,...,2137.25,2137.25,2137.25,2137.25,2137.25,2137.25,2137.25,1.75,1.75


In [ ]:
db=prj.getstaffcosts()
# # Pivot the DataFrame
db.pivot_table(index='item', columns='name', values='budget', aggfunc='sum', fill_value=0)
# type(prj.policies[1])==FullCostRecovery
# pivotbudget(db)

name,Bubu,Olaf
item,,
Finance set-up & Budget import,315.00,315.00
"HR (payroll, timesheets, performance management)",1680.00,1680.00
ICT procurement & setup,350.00,350.00
ICT support,300.00,300.00
Line Management,9600.00,9600.00
Recruitment,980.00,980.00
ni,3912.51,5268.22
pension,4194.14,5078.30
salary,46601.50,56425.50


In [ ]:
prj.income

45000